In [1]:
import numpy as np
import pandas as pd
import sympy

# Proof of Concept on Fictional Toy Data
Three time resolutions, one state, one fuel, one sector. A single year.

Monthly:
* Q1 has no unknowns
* Q2 has one unknown month
* Q2 has two unknown months
* Q3 has three unknown months

Quarterly
* Q1 is unknown
* Q2 is known
* Q3 is known
* Q4 is unknown

Annual is known.

Should be able to solve for:
* Month June 2015 via Q2 total
* Q1 total via known months
* Q4 total via known Q2, Q3 and derived Q1

Should NOT be able to solve for:
* any missing months in Q3 and Q4

## Make Toy Data

In [175]:
# made in and copied from a spreadsheet
data = """
geography,fuel,sector,time_resolution,timestamp,price,quantity
AK,gas,utility,monthly,01-2015,5,100
AK,gas,utility,monthly,02-2015,4,100
AK,gas,utility,monthly,03-2015,6,100
AK,gas,utility,monthly,04-2015,3,100
AK,gas,utility,monthly,05-2015,2,100
AK,gas,utility,monthly,06-2015,NA,100
AK,gas,utility,monthly,07-2015,7,100
AK,gas,utility,monthly,08-2015,NA,100
AK,gas,utility,monthly,09-2015,NA,100
AK,gas,utility,monthly,10-2015,NA,100
AK,gas,utility,monthly,11-2015,NA,100
AK,gas,utility,monthly,12-2015,NA,100
AK,gas,utility,quarterly,01-2015,NA,300
AK,gas,utility,quarterly,04-2015,2,300
AK,gas,utility,quarterly,07-2015,6,300
AK,gas,utility,quarterly,10-2015,NA,300
AK,gas,utility,annual,01-2015,4.5,1200
"""

In [7]:
from io import StringIO

In [176]:
df = pd.read_csv(StringIO(data), parse_dates=['timestamp'])

In [177]:
df['total'] = df['quantity'] * df['price']

In [178]:
df

,geography,fuel,sector,time_resolution,timestamp,price,quantity,total
0,AK,gas,utility,monthly,2015-01-01,5.0,100,500.0
1,AK,gas,utility,monthly,2015-02-01,4.0,100,400.0
2,AK,gas,utility,monthly,2015-03-01,6.0,100,600.0
3,AK,gas,utility,monthly,2015-04-01,3.0,100,300.0
4,AK,gas,utility,monthly,2015-05-01,2.0,100,200.0
5,AK,gas,utility,monthly,2015-06-01,NaN,100,NaN
6,AK,gas,utility,monthly,2015-07-01,7.0,100,700.0
7,AK,gas,utility,monthly,2015-08-01,NaN,100,NaN
8,AK,gas,utility,monthly,2015-09-01,NaN,100,NaN
9,AK,gas,utility,monthly,2015-10-01,NaN,100,NaN


### Convert to matrix form
Ax = b, or Quantity \* price = total

The key part is to represent all the overlapping information: each aggregate is both its own value and a sum of other aggregates. That means duplicating the values for each equation.

In [37]:
np.set_printoptions(edgeitems=30, linewidth=100000,)

In [179]:
values = np.diagflat(df['quantity'].to_numpy())
values

array([[ 100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,  100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,  100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,  100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,  100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,  100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,  100,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,  100,    0,    0,    0,    0,    0,    0,    0,    0],
 

#### Add hierarchical relationship equations
This is the key piece that models the hierarchical relationships, even in the absence of measured values. This works because this row is always == 0, regardless of data availability.

In [170]:
quarterly_monthly_rel = np.array([[0]*3*i+[100]*3+[0]*3*(3-i)+[0]*i+[-300]+[0]*(3-i)+[0] for i in range (4)])
quarterly_monthly_rel

array([[ 100,  100,  100,    0,    0,    0,    0,    0,    0,    0,    0,    0, -300,    0,    0,    0,    0],
       [   0,    0,    0,  100,  100,  100,    0,    0,    0,    0,    0,    0,    0, -300,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,  100,  100,  100,    0,    0,    0,    0,    0, -300,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,  100,  100,  100,    0,    0,    0, -300,    0]])

In [171]:
annual_monthly_rel = np.array([100]*12 + [0]*4 + [-1200])
annual_monthly_rel

array([  100,   100,   100,   100,   100,   100,   100,   100,   100,   100,   100,   100,     0,     0,     0,     0, -1200])

In [173]:
annual_quarterly_rel = np.array([0]*12 + [300]*4 + [-1200])
annual_quarterly_rel

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,   300,   300,   300,   300, -1200])

In [180]:
A = np.vstack([values, quarterly_monthly_rel, annual_monthly_rel, annual_quarterly_rel])
A

array([[  100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,   100,     0,     0,     0,     0,     0,     0

In [185]:
b = np.vstack([
    df['total'].to_numpy().reshape(-1,1), # data values
    np.zeros((6,1)), # relationship equations (zeros)
])
b

array([[ 500.],
       [ 400.],
       [ 600.],
       [ 300.],
       [ 200.],
       [  nan],
       [ 700.],
       [  nan],
       [  nan],
       [  nan],
       [  nan],
       [  nan],
       [  nan],
       [ 600.],
       [1800.],
       [  nan],
       [5400.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.]])

In [186]:
not_nan = ~np.isnan(b).reshape(-1)

In [187]:
b = b[not_nan]
b

array([[ 500.],
       [ 400.],
       [ 600.],
       [ 300.],
       [ 200.],
       [ 700.],
       [ 600.],
       [1800.],
       [5400.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.]])

In [188]:
A = A[not_nan, :]
A

array([[  100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,   100,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,   300,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0

x is the rows of df

## Solve

In [189]:
system = (sympy.Matrix(A), sympy.Matrix(b))

In [190]:
solution = sympy.solvers.linsolve(system)

In [191]:
solution

{(5.0, 4.0, 6.0, 3.0, 2.0, 1.0, 7.0, 11.0 - 1.0*tau0, tau0, -1.0*tau1 - 1.0*tau2 + 15.0, tau1, tau2, 5.0, 2.0, 6.0, 5.0, 4.5)}

In [193]:
df['solution'] = list(*solution)
df

,geography,fuel,sector,time_resolution,timestamp,price,quantity,total,solution
0,AK,gas,utility,monthly,2015-01-01,5.0,100,500.0,5.00000000000000
1,AK,gas,utility,monthly,2015-02-01,4.0,100,400.0,4.00000000000000
2,AK,gas,utility,monthly,2015-03-01,6.0,100,600.0,6.00000000000000
3,AK,gas,utility,monthly,2015-04-01,3.0,100,300.0,3.00000000000000
4,AK,gas,utility,monthly,2015-05-01,2.0,100,200.0,2.00000000000000
5,AK,gas,utility,monthly,2015-06-01,NaN,100,NaN,1.00000000000000
6,AK,gas,utility,monthly,2015-07-01,7.0,100,700.0,7.00000000000000
7,AK,gas,utility,monthly,2015-08-01,NaN,100,NaN,11.0 - 1.0*tau0
8,AK,gas,utility,monthly,2015-09-01,NaN,100,NaN,tau0
9,AK,gas,utility,monthly,2015-10-01,NaN,100,NaN,-1.0*tau1 - 1.0*tau2 + 15.0


In [195]:
type(list(*solution)[0])

sympy.core.numbers.Float

Success! All three recoverable values were obtained.

Compare to other solvers:
#### Numpy least squares
The problem with this is it gives a particular solution for underdetermined systems. How can I tell which variables are solved exactly and which are arbitrary?

In [204]:
x1, resid, rnk, s = np.linalg.lstsq(A,b)
x1

/tmp/ipykernel_2930/1872178327.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x1, resid, rnk, s = np.linalg.lstsq(A,b)


array([[5. ],
       [4. ],
       [6. ],
       [3. ],
       [2. ],
       [1. ],
       [7. ],
       [5.5],
       [5.5],
       [5. ],
       [5. ],
       [5. ],
       [5. ],
       [2. ],
       [6. ],
       [5. ],
       [4.5]])

In [205]:
rnk

14

In [206]:
resid

array([], dtype=float64)

#### Scipy NNLS

In [213]:
from scipy.optimize import nnls

In [216]:
x2, resid = nnls(A,b.reshape(-1))

In [220]:
np.hstack([x1, x2.reshape(-1,1)])

array([[ 5. ,  5. ],
       [ 4. ,  4. ],
       [ 6. ,  6. ],
       [ 3. ,  3. ],
       [ 2. ,  2. ],
       [ 1. ,  1. ],
       [ 7. ,  7. ],
       [ 5.5,  0. ],
       [ 5.5, 11. ],
       [ 5. ,  0. ],
       [ 5. , 15. ],
       [ 5. ,  0. ],
       [ 5. ,  5. ],
       [ 2. ,  2. ],
       [ 6. ,  6. ],
       [ 5. ,  5. ],
       [ 4.5,  4.5]])

Can use two different solvers and check for differences. There must be a better way?

In [222]:
np.abs(x1.reshape(-1) - x2) < 1e-10

array([ True,  True,  True,  True,  True,  True,  True, False, False, False, False, False,  True,  True,  True,  True,  True])

#### Find the Null Space numerically
Non-zero (numerically speaking) coefficients in the null space show which variables are undetermined. But this requires a second decomposition.

In [223]:
from scipy.linalg import qr

def qr_null(A, tol=None):
    """Computes the null space of A using a rank-revealing QR decomposition"""
    Q, R, P = qr(A.T, mode='full', pivoting=True)
    tol = np.finfo(R.dtype).eps if tol is None else tol
    rnk = min(A.shape) - np.abs(np.diag(R))[::-1].searchsorted(tol)
    return Q[:, rnk:].conj()

In [246]:
# Compute the null space of `A`
Z = qr_null(A)
nullity = Z.shape[1]
nullity

3

In [260]:
# Sample some random solutions
for _ in range(1):
    x_rand = x1 + Z.dot(np.random.rand(nullity)).reshape(-1,1)
    # If `x_rand` is a solution then `||A·x_rand - b||` should be very small
    out = A.dot(x_rand)
    error = np.linalg.norm(out - b)
    print(x_rand, error)

[[5.        ]
 [4.        ]
 [6.        ]
 [3.        ]
 [2.        ]
 [1.        ]
 [7.        ]
 [5.13192502]
 [5.86807498]
 [5.81919635]
 [4.65161849]
 [4.52918516]
 [5.        ]
 [2.        ]
 [6.        ]
 [5.        ]
 [4.5       ]] 5.7330092970254436e-11


In [261]:
# Non-zero (numerically speaking) coefficients in the null space show which variables are undetermined.
(np.abs(Z[:,0]) > 1e-12).reshape(-1,1)

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False]])